<a href="https://colab.research.google.com/github/dvschultz/SRFBN_CVPR19/blob/master/SRFBN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Paper99/SRFBN_CVPR19.git
%cd SRFBN_CVPR19/models
!gdown --id 1Dsb_-OH0CeSJVjvP9A4bh2_IBQh9R-ja
!unzip SRFBN_CVPR19_Models.zip
!gdown --id 1NYIS0zPrpSO5fv1xpN1hH6qX4lBYxIW5
!unzip SRFBN-S_CVPR19_Models.zip
!rm /content/SRFBN_CVPR19/models/SRFBN-S_CVPR19_Models.zip
!rm /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models.zip
%cd ../
mv /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models/*.pth /content/SRFBN_CVPR19/models/
rmdir /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models

Cloning into 'SRFBN_CVPR19'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 218 (delta 0), reused 2 (delta 0), pack-reused 212
Receiving objects: 100% (218/218), 3.51 MiB | 33.94 MiB/s, done.
Resolving deltas: 100% (70/70), done.
/content/SRFBN_CVPR19/models
Downloading...
From: https://drive.google.com/uc?id=1Dsb_-OH0CeSJVjvP9A4bh2_IBQh9R-ja
To: /content/SRFBN_CVPR19/models/SRFBN_CVPR19_Models.zip
52.9MB [00:00, 145MB/s] 
Archive:  SRFBN_CVPR19_Models.zip
   creating: SRFBN_CVPR19_Models/
  inflating: SRFBN_CVPR19_Models/SRFBN_x2_BI.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x3_DN.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x3_BD.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x3_BI.pth  
  inflating: SRFBN_CVPR19_Models/SRFBN_x4_BI.pth  
Downloading...
From: https://drive.google.com/uc?id=1NYIS0zPrpSO5fv1xpN1hH6qX4lBYxIW5
To: /content/SRFBN_CVPR19/models/SRFBN-S_CVPR19_Models.zip
4.25MB 

In [0]:
import argparse, time, os
import imageio

import options.options as option
from utils import util
from solvers import create_solver
from data import create_dataloader
from data import create_dataset

def run():
  opt = option.parse('/content/SRFBN_CVPR19/options/test/test_SRFBN_example.json')
  opt = option.dict_to_nonedict(opt)
  print(opt)

  # initial configure
  scale = opt['scale']
  degrad = opt['degradation']
  network_opt = opt['networks']
  model_name = network_opt['which_model'].upper()
  if opt['self_ensemble']: model_name += 'plus'

  # create test dataloader
  bm_names =[]
  test_loaders = []
  for _, dataset_opt in sorted(opt['datasets'].items()):
      test_set = create_dataset(dataset_opt)
      test_loader = create_dataloader(test_set, dataset_opt)
      test_loaders.append(test_loader)
      print('===> Test Dataset: [%s]   Number of images: [%d]' % (test_set.name(), len(test_set)))
      bm_names.append(test_set.name())

  # create solver (and load model)
  solver = create_solver(opt)
  # Test phase
  print('===> Start Test')
  print("==================================================")
  print("Method: %s || Scale: %d || Degradation: %s"%(model_name, scale, degrad))

  for bm, test_loader in zip(bm_names, test_loaders):
      print("Test set : [%s]"%bm)

      sr_list = []
      path_list = []

      total_psnr = []
      total_ssim = []
      total_time = []

      need_HR = False

      for iter, batch in enumerate(test_loader):
          solver.feed_data(batch, need_HR=need_HR)

          # calculate forward time
          t0 = time.time()
          solver.test()
          t1 = time.time()
          total_time.append((t1 - t0))

          visuals = solver.get_current_visual(need_HR=need_HR)
          sr_list.append(visuals['SR'])

          # calculate PSNR/SSIM metrics on Python
          if need_HR:
              psnr, ssim = util.calc_metrics(visuals['SR'], visuals['HR'], crop_border=scale)
              total_psnr.append(psnr)
              total_ssim.append(ssim)
              path_list.append(os.path.basename(batch['HR_path'][0]).replace('HR', model_name))
              print("[%d/%d] %s || PSNR(dB)/SSIM: %.2f/%.4f || Timer: %.4f sec ." % (iter+1, len(test_loader),
                                                                                      os.path.basename(batch['LR_path'][0]),
                                                                                      psnr, ssim,
                                                                                      (t1 - t0)))
          else:
              path_list.append(os.path.basename(batch['LR_path'][0]))
              print("[%d/%d] %s || Timer: %.4f sec ." % (iter + 1, len(test_loader),
                                                          os.path.basename(batch['LR_path'][0]),
                                                          (t1 - t0)))

      if need_HR:
          print("---- Average PSNR(dB) /SSIM /Speed(s) for [%s] ----" % bm)
          print("PSNR: %.2f      SSIM: %.4f      Speed: %.4f" % (sum(total_psnr)/len(total_psnr),
                                                                sum(total_ssim)/len(total_ssim),
                                                                sum(total_time)/len(total_time)))
      else:
          print("---- Average Speed(s) for [%s] is %.4f sec ----" % (bm,
                                                                    sum(total_time)/len(total_time)))

      # save SR results for further evaluation on MATLAB
      if need_HR:
          save_img_path = os.path.join('./results/SR/'+degrad, model_name, bm, "x%d"%scale)
      else:
          save_img_path = os.path.join('./results/SR/'+bm, model_name, "x%d"%scale)

      print("===> Saving SR images of [%s]... Save Path: [%s]\n" % (bm, save_img_path))

      if not os.path.exists(save_img_path): os.makedirs(save_img_path)
      for img, name in zip(sr_list, path_list):
          imageio.imwrite(os.path.join(save_img_path, name), img)

  print("==================================================")
  print("===> Finished !")


In [33]:
run()

===> Export CUDA_VISIBLE_DEVICES = [0]
{'mode': 'sr', 'use_cl': True, 'gpu_ids': [0], 'scale': 4, 'degradation': 'BI', 'is_train': False, 'use_chop': True, 'rgb_range': 255, 'self_ensemble': False, 'datasets': {'test_set1': {'mode': 'LRHR', 'dataroot_HR': './results/HR/Set5/x4', 'dataroot_LR': './results/LR/LRBI/Set5/x4', 'data_type': 'img', 'phase': 'test', 'scale': 4, 'rgb_range': 255}, 'test_set3': {'mode': 'LR', 'dataroot_LR': './results/LR/MyImage', 'data_type': 'img', 'phase': 'test', 'scale': 4, 'rgb_range': 255}}, 'networks': {'which_model': 'SRFBN', 'num_features': 64, 'in_channels': 3, 'out_channels': 3, 'num_steps': 4, 'num_groups': 6, 'scale': 4}, 'solver': {'pretrained_path': './models/SRFBN_x4_BI.pth'}, 'timestamp': '200208-012900', 'path': {'exp_root': 'experiments/SRFBN_in3f64_x4', 'epochs': 'experiments/SRFBN_in3f64_x4/epochs', 'visual': 'experiments/SRFBN_in3f64_x4/visual', 'records': 'experiments/SRFBN_in3f64_x4/records'}}
===> [LRHR] Dataset is created.
===> Test Da

In [34]:
!zip -r saved.zip /content/SRFBN_CVPR19/results/LR/MyImage

  adding: content/SRFBN_CVPR19/results/LR/MyImage/ (stored 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04230.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04208.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04239.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04215.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04232.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04224.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04234.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04212.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04210.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04220.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04227.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage/04240.png (deflated 0%)
  adding: content/SRFBN_CVPR19/results/LR/MyImage

In [0]:
rm /content/SRFBN_CVPR19/results/HR/Set5/x2/*.png